In [ ]:
import os.path as osp
import json
import csv

import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from torcheval.metrics import MeanSquaredError as MSE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from PlayerPerformancePredictors import GruPlayerPredictor as GRU
from GruTraining import get_games_window, train_gru

ModuleNotFoundError: No module named 'models'

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)

Using device cuda


In [7]:
# Load and clean data
player_stats_per_game_path = osp.join('backend','json', 'nba_data.json')
with open(player_stats_per_game_path, 'r') as fd:
    player_stats_per_game = json.load(fd)

X, y = get_games_window(player_stats_per_game, time_minimum=12)



In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)

print("Input Shape:", X.shape, "Target Shape:", y.shape)
print("Training Set Size:", len(X_train), 
      "Testing Set Size:", len(X_test), 
      "Total:", len(X)
)
print("Ex input:", X_train[0])
print("Ex target:", y_train[0])

Input Shape: torch.Size([42388, 3, 20]) Target Shape: torch.Size([42388, 20])
Training Set Size: 33910 Testing Set Size: 8478 Total: 42388
Ex input: tensor([[33.0000,  8.0000, 10.0000,  0.8000,  0.0000,  1.0000,  0.0000,  3.0000,
          4.0000,  0.7500,  5.0000,  8.0000, 13.0000,  2.0000,  1.0000,  1.0000,
          0.0000,  1.0000, 19.0000,  7.0000],
        [27.0000,  5.0000,  7.0000,  0.7140,  0.0000,  0.0000,  0.0000,  1.0000,
          4.0000,  0.2500,  3.0000,  5.0000,  8.0000,  1.0000,  0.0000,  2.0000,
          1.0000,  2.0000, 11.0000, -5.0000],
        [31.0000,  7.0000,  9.0000,  0.7780,  0.0000,  0.0000,  0.0000,  2.0000,
          3.0000,  0.6670,  7.0000,  6.0000, 13.0000,  0.0000,  0.0000,  2.0000,
          1.0000,  1.0000, 16.0000, -4.0000]])
Ex target: tensor([30.0000,  8.0000, 10.0000,  0.8000,  0.0000,  0.0000,  0.0000,  1.0000,
         4.0000,  0.2500,  9.0000,  6.0000, 15.0000,  1.0000,  0.0000,  2.0000,
         1.0000,  0.0000, 17.0000, -2.0000])


In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.view(-1, 20))
X_train = torch.tensor(X_train, dtype=torch.float, device=device).view(-1, 3, 20)
y_train = scaler.transform(y_train)
y_train = torch.column_stack([i for i in torch.tensor(y_train, dtype=torch.float, device=device)])

X_test = scaler.transform(X_test.view(-1, 20))
X_test = torch.tensor(X_test, dtype=torch.float, device=device).view(-1, 3, 20)
y_test = scaler.transform(y_test)
y_test = torch.column_stack([i for i in torch.tensor(y_test, dtype=torch.float, device=device)])

In [24]:
# Define parameters
# Training parameters
num_epochs = 30
learning_rate = .001
batch_size = 8

# Model parameters
input_size = 20
gru_hidden_size = 32
gru_num_layers = 4
linear_output_size = 1

In [ ]:
model_folder = "models/models"
metrics_folder = "models/metrics"
target_features = [
    'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
    'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO',
    'PF', 'PTS', 'PLUS_MINUS'
]
calculated_features = ['FG_PCT', 'FG3_PCT', 'FT_PCT']

torch.manual_seed(32)
for feature_index, feature in enumerate(target_features):
    if feature in calculated_features:
        continue

    train_dataset = TensorDataset(X_train, y_train[feature_index].unsqueeze(dim=1))
    test_dataset = TensorDataset(X_test, y_test[feature_index].unsqueeze(dim=1))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    model = GRU(
        input_size,
        gru_hidden_size,
        linear_output_size,
        gru_num_layers=gru_num_layers
    )

    model.to(device)
    opt = Adam(model.parameters(), lr=learning_rate)

    print("Training GRU for feature", feature)
    model_dict, model_metrics = train_gru(
        model, 
        opt, 
        train_loader,
        test_loader,
        num_epochs,
        device
    )
    
    torch.save(model_dict, osp.join(model_folder, f"{feature}_gru.pt"))
    with open(osp.join(metrics_folder, f"{feature}_gru_metrics.csv"), "w", newline="") as fd:
        writer = csv.writer(fd)
        writer.writerows(model_metrics)

    print(f"Successfully trained and stored {feature} GRU")
    


Training GRU for feature MIN


KeyboardInterrupt: 

In [34]:
from joblib import dump
dump(scaler, 'models/models16/std_scaler_gru.gz')

['models/models16/std_scaler_gru.gz']